# Trajectory optimization

In [ ]:
using Plots;
gr();
@time using FUSE
FUSE.logging(Logging.Info; actors=Logging.Info);
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
# take the ITER hardware configuration from ods
ini, act = FUSE.case_parameters(:ITER; init_from=:ods);
plot(ini)

In [ ]:
# # this is useful to see changes in the equilibrium
# using Interact
# @manipulate for time0 in ini.time.pulse_shedule_time_basis
#     plot(ini; color=:red)
#     plot!(ini; time0, color=:blue)
# end

In [ ]:
# initialize hardware
chk = FUSE.Checkpoint();
dd = IMAS.dd()
@time dd = FUSE.init(dd, ini, act);
chk[:init] = dd, ini, act;

In [ ]:
dd, ini, act = chk[:init]

# switch to scalars and wind back simulation time
ini, act = FUSE.case_parameters(:ITER; init_from=:scalars);
ini.time.simulation_start = 100.0

# FUSE.SimulationParameters.rand!(ini.equilibrium, :ip)
# FUSE.SimulationParameters.rand!(ini.ec_launcher[1], :rho_0)

FUSE.init(dd, ini, act; initialize_hardware=false);

display(plot(ini))
chk[:tinit] = (dd, ini, act);

In [ ]:
# make sure time dependence has been transferred from `ini` to `dd.pulse_schedule`
using Interact
@manipulate for time0 in dd.pulse_schedule.time
    plot(dd.pulse_schedule; time0)
end

In [ ]:
# flux matcher
dd, ini, act = chk[:tinit]

act.ActorCoreTransport.model = :FluxMatcher

# act.ActorStationaryPlasma.convergence_error=1E-3
# act.ActorStationaryPlasma.max_iter=3
# act.ActorFluxMatcher.max_iterations = 50
# act.ActorFluxMatcher.rho_transport=0.25:0.1:0.85
# act.ActorFluxMatcher.optimizer_algorithm=:trust_region

act.ActorFluxMatcher.step_size = 1.0
act.ActorFluxMatcher.optimizer_algorithm = :anderson
act.ActorFluxMatcher.max_iterations = 300
act.ActorStationaryPlasma.verbose = true;

FUSE.ActorStationaryPlasma(dd, act; do_plot=false, verbose=true)

chk[:stationary] = (dd, ini, act);

In [ ]:
# estimate q profile assuming breakdown happened Δt seconds back
dd, ini, act = chk[:stationary]

act.ActorQED.Δt = -60.0
actor = FUSE.ActorCurrent(dd, act; ip_from=:pulse_schedule, vloop_from=:pulse_schedule, model=:QED)

chk[:stationary_rampup] = (dd, ini, act);

In [ ]:
FUSE.plot_plasma_overview(dd, dd.global_time; min_power=1E6, aggregate_radiation=true)

In [ ]:
# run time evolution
FUSE.ProgressMeter.ijulia_behavior(:clear)

dd, ini, act = chk[:stationary_rampup]

act.ActorFluxMatcher.max_iterations = 50
act.ActorFluxMatcher.rho_transport = 0.25:0.1:0.85
act.ActorFluxMatcher.optimizer_algorithm = :trust_region

act.ActorDynamicPlasma.Nt = 30
act.ActorDynamicPlasma.Δt = 200.0
act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = false
act.ActorDynamicPlasma.evolve_transport = false
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = false
act.ActorDynamicPlasma.verbose=true

chk[:stationary_rampup_w_act] = dd, ini, act

FUSE.ActorDynamicPlasma(dd, act);

chk[:time_dep] = dd, ini, act;

In [ ]:
dd.global_time = 200.0
lw = 2
using Interact

a = @animate for (k, time0) in enumerate(dd.core_sources.time)
    #@manipulate for k in eachindex(dd.core_sources.time)
    time0 = dd.core_sources.time[k]
    FUSE.plot_plasma_overview(dd, time0; min_power=1E6, aggregate_radiation=true)
    #    savefig(joinpath("/Users/meneghini/Library/CloudStorage/Dropbox/tex/figures/FUSE/gif", "frame_$(lpad(k-1, 4, '0')).png"));
end

In [ ]:
g = gif(a, "ITER_time_dep_v5.gif", fps=12)
display(g)

In [ ]:
dd0, ini, act = chk[:stationary_rampup_w_act]

function my_workflow(dd, ini, act)
    FUSE.init_pulse_schedule!(dd, ini, act);    
    FUSE.ActorDynamicPlasma(dd, act)
    return dd
end

objective_functions = FUSE.ObjectiveFunction[]
push!(objective_functions,
    FUSE.ObjectiveFunction(:flat_q, "-", 
    dd -> begin
            eqt1d = dd.equilibrium.time_slice[].profiles_1d
            index = eqt1d.rho_tor_norm .< 0.8
            m = sum(eqt1d.q[index]) / length(index)
             return sum(((eqt1d.q[index].-m) / m).^2)
        end,
    -Inf))

FUSE.workflow_multiobjective_optimization(
    ini,
    act,
    (ini, act) -> my_workflow(deepcopy(dd0), ini, act),
    objective_functions,
    FUSE.ConstraintFunction[];
    exploitation_vs_exploration=0.0,
    N=2,
    iterations=2,
    continue_state=nothing,
    save_folder="optimization_runs",
    save_dd=true)

In [ ]:
pwd()

In [ ]:
# issue with expressions and refs

dd, ini, act = chk[:stationary]

tt = dd.global_time + 1.0
IMAS.new_timeslice!(dd.core_profiles, tt)
IMAS.new_timeslice!(dd.equilibrium, tt)
dd.global_time = tt

act.ActorQED.Δt=100.0
display(act.ActorQED)

actor=FUSE.ActorCurrent(dd,act;ip_from=:pulse_schedule, vloop_from=:pulse_schedule, model=:QED)

#[IMAS.integrate(dd.core_profiles.profiles_1d[tt].grid.area, dd.core_profiles.profiles_1d[tt].j_non_inductive) for tt in dd.core_profiles.time]
plot([dd.core_profiles.profiles_1d[tt].j_total for tt in dd.core_profiles.time])
empty!(dd.core_profiles.profiles_1d[tt], :j_tor)
#empty!(dd.core_profiles.profiles_1d[1], :j_tor)
dd.core_profiles.profiles_1d[tt]
@show dd.core_profiles.global_quantities.ip